In [ ]:
#input blog link: https://medium.com/@jainashish.079/build-llm-agent-combining-reasoning-and-action-react-framework-using-langchain-379a89a7e881

In [1]:
import os 
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chains.retrieval import create_retrieval_chain

from dotenv import find_dotenv, load_dotenv, dotenv_values
load_dotenv('../.env.secret')

True

In [2]:
# load text
loader = TextLoader('blog.txt')
document = loader.load()

# split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50)

documents = text_splitter.split_documents(document)
print(len(documents))

# Embedding text and storing it in vectore store
vector_db = Chroma.from_documents(documents, OpenAIEmbeddings(), persist_directory="./chroma_db_blog")

35


In [3]:
query = 'What are the key findings of the latest research on quantum computing published in 2024'
llm = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_template(template=query)
chain = prompt | llm
result = chain.invoke(input={"input": query})

In [4]:
print(result.content)

1. Breakthrough in error correction: Researchers have made significant progress in developing error correction techniques for quantum computers, which is crucial for overcoming the inherent fragility of quantum systems.

2. Scalability: The latest research has shown promising results in scaling up quantum computing systems, with the potential to handle larger and more complex problems.

3. Quantum supremacy: There have been reports of achieving quantum supremacy in certain tasks, demonstrating the superiority of quantum computers over classical computers in specific applications.

4. Quantum algorithms: New quantum algorithms have been developed that show improved performance compared to classical algorithms, opening up new possibilities for solving complex problems efficiently.

5. Quantum networking: Progress has been made in developing quantum networks that can securely transmit information over long distances, paving the way for a quantum internet.

6. Quantum machine learning: Res

## RAG using langchain

In [70]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", """Answer any use questions based solely on the context below:

    <context>
    {context}
    </context>"""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
])

In [71]:
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_db.as_retriever()

combine_docs_chain = create_stuff_documents_chain(llm, qa_prompt)
retrival_chain = create_retrieval_chain(
    retriever=retriever, combine_docs_chain=combine_docs_chain
)

In [78]:
# code for sample retrival
query = 'explain me the example given in the blog'
result = retrival_chain.invoke(input={"input": query})

print(result)
print('\n\n',result['answer'])

{'input': 'explain me the example given in the blog', 'context': [Document(metadata={'source': 'blog.txt'}, page_content='as the one-shot example. By structuring the examples in this way, we are essentially teaching the model how to reason through the task to reach a solution. The model now correctly determines that nine apples are left. Thinking through the problem has helped the model come to the correct answer. We can use chain of thought prompting to help LLMs improve their reasoning for other types of problems.'), Document(metadata={'source': 'blog.txt'}, page_content='ReAct Prompt Template\nWe can come up with standard template for ReAct for queries. We are asking question with LLM inside the prompt with instruction and some examples.\n\n\nReAct Execution Flow\nModel can start its execution by learning from the given example. It can start its process by taking first thought.\n\nThought1 I need to search Amitabh Bachchan and Shahrukh Khan, find their profession, then find the prof

## Custom RAG chain

In [59]:
template="""
Use the following context to answer the question at the end.
If you don't know the answer then just say I don't know the answer, dont make up any other answer.
Use the 3 sentence at max and keep the answer simple and short.
Always say "Let me know if you need any other help!"

<context>
{context}
</context>

Question: {question}
Helpful Answer:
"""

from langchain.prompts import PromptTemplate
custom_rag_prompt = PromptTemplate(template=template)
print(custom_rag_prompt)

input_variables=['context', 'question'] template='\nUse the following context to answer the question at the end.\nIf you don\'t know the answer then just say I don\'t know the answer, dont make up any other answer.\nUse the 3 sentence at max and keep the answer simple and short.\nAlways say "Let me know if you need any other help!"\n\n<context>\n{context}\n</context>\n\nQuestion: {question}\nHelpful Answer:\n'


In [60]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
retriever = vector_db.as_retriever()


In [61]:
def format_docs(docs):
    return '\n\n'.join([i.page_content for i in docs])

custom_chain = {'context':retriever | format_docs, 'question':RunnablePassthrough()} | custom_rag_prompt | llm

AIMessage(content="The example in the blog demonstrates how a language model reasons through a task to reach a solution. By structuring examples in a certain way, the model can correctly determine the answer. This approach helps improve the model's reasoning abilities for various types of problems. Let me know if you need any other help!", response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 412, 'total_tokens': 473}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-410b21a4-b5ff-4442-8719-45c3c6c2eb00-0', usage_metadata={'input_tokens': 412, 'output_tokens': 61, 'total_tokens': 473})

In [ ]:
query = 'explain me the example given in the blog'
result = custom_chain.invoke(query)
result

In [62]:
print(result.content)

The example in the blog demonstrates how a language model reasons through a task to reach a solution. By structuring examples in a certain way, the model can correctly determine the answer. This approach helps improve the model's reasoning abilities for various types of problems. Let me know if you need any other help!
